# search parameter space for reasonable to allow 1C discharge then 0.3C charge: Xi, t_0plus,D_cross, D_ec 

In [1]:
import pybamm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;
from scipy.io import savemat,loadmat;from pybamm import constants,exp,tanh,sqrt;import matplotlib as mpl; fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
for k in range(0,1):
    mpl.rcParams["axes.labelsize"] = fs
    mpl.rcParams["axes.titlesize"] = fs
    mpl.rcParams["xtick.labelsize"] =  fs
    mpl.rcParams["ytick.labelsize"] =  fs
    mpl.rcParams["legend.fontsize"] =  fs
    mpl.rcParams['font.sans-serif'] = ['Times New Roman']
    mpl.rcParams['font.serif'] = ['Times New Roman']
    mpl.rcParams['axes.unicode_minus'] = False
import traceback

In [3]:
V_max = 4.2;        V_min = 2.5;
Diff_SEI = 1.7e-20;    R_SEI = 2E5;   Bulk_Sol_Con = 4541.0;
Experiment_Long   = pybamm.Experiment( [ (
    f"Discharge at 1 C until {V_min} V", 
    f"Charge at 0.3 C until {V_max} V", 
    f"Hold at {V_max} V until C/5")    ] * 1 ) 
Experiment_s   = pybamm.Experiment( [ (
    f"Discharge at 1 C for 240 s", )    ] * 1 )   

In [4]:
BasicPath = 'D:/OneDrive - Imperial College London/SimDataSave/P3R4/'; 
Target  = 'a2_Diff_Cond_c_EC/'
if not os.path.exists(BasicPath + Target):
    os.mkdir(BasicPath + Target);

In [5]:
Model_ddiff = pybamm.lithium_ion.DFN(     
    options={"solvent diffusion": "EC wo refill"     } )
Model_sdiff = pybamm.lithium_ion.DFN(     
    options={"solvent diffusion": "none"     } )
ChemistryChen=pybamm.parameter_sets.Chen2020 
ChemistryChen["electrolyte"] = "lipf6_Nyman2008_rio";
Para_0=pybamm.ParameterValues(chemistry=ChemistryChen);


Para_0['EC Lithium ion cross diffusivity [m2.s-1]'] = "[input]"
Para_0['Typical EC Lithium ion cross diffusivity [m2.s-1]'] =   1.5e-9/1e2
Para_0['EC transference number'] =     "[input]" # -1.4
Para_0['EC diffusivity in electrolyte [m2.s-1]'] =   "[input]" # 5E-12;
c_e_ddiff = Model_ddiff.variables["Electrolyte concentration [mol.m-3]"]
T_ddiff = Model_ddiff.variables["Cell temperature [K]"]
c_EC_ddiff = Model_ddiff.variables["EC concentration [mol.m-3]"]
Model_ddiff.variables["c(EC) over c(Li+)"] = c_EC_ddiff / c_e_ddiff
Model_ddiff.variables["Electrolyte conductivity [S.m-1]"] =Para_0['Electrolyte conductivity [S.m-1]'](c_e_ddiff,c_EC_ddiff, T_ddiff)
Model_ddiff.variables["Electrolyte diffusivity [m2.s-1]"] =Para_0['Electrolyte diffusivity [m2.s-1]'](c_e_ddiff,c_EC_ddiff, T_ddiff)

using EC wo refill for Li+


Xi =  list(np.linspace(-1.8, -1.0, 10))
print(type(Xi))

In [6]:
Sim_ddiff    = pybamm.Simulation(
    Model_ddiff, experiment = Experiment_Long,
    parameter_values = Para_0,
    solver = pybamm.CasadiSolver(),)   
Sol_ddiff    = Sim_ddiff.solve(   # andrew results
    inputs={
        "EC transference number":-1.4,
        "Cation transference number":0.28,
        "EC Lithium ion cross diffusivity [m2.s-1]":5e-14,
        "EC diffusivity in electrolyte [m2.s-1]":5e-10,});

In [18]:
Sol_ddiff2    = Sim_ddiff.solve(
    inputs={
        "EC transference number":-1.4,
        "Cation transference number":0.28,
        "EC Lithium ion cross diffusivity [m2.s-1]":5e-12,
        "EC diffusivity in electrolyte [m2.s-1]":5e-10,});
Sol_ddiff3    = Sim_ddiff.solve(
    inputs={
        "EC transference number":-1.4,
        "Cation transference number":0.28,
        "EC Lithium ion cross diffusivity [m2.s-1]":5e-11,
        "EC diffusivity in electrolyte [m2.s-1]":5e-10,});

In [29]:
Sol_ddiff6    = Sim_ddiff.solve(
    inputs={
        "EC transference number":-1.4,
        "Cation transference number":0.28,
        "EC Lithium ion cross diffusivity [m2.s-1]":1.5e-11,
        "EC diffusivity in electrolyte [m2.s-1]":5e-12,});

In [19]:
Sol_ddiff4    = Sim_ddiff.solve(
    inputs={
        "EC transference number":-1.4,
        "Cation transference number":0.28,
        "EC Lithium ion cross diffusivity [m2.s-1]":5e-10,
        "EC diffusivity in electrolyte [m2.s-1]":5e-10,});

In [20]:
Sol_ddiff5    = Sim_ddiff.solve(
    inputs={
        "EC transference number":-1.4,
        "Cation transference number":0.28,
        "EC Lithium ion cross diffusivity [m2.s-1]":1.5e-9,
        "EC diffusivity in electrolyte [m2.s-1]":5e-10,});

At t = 0.00408377, , mxstep steps taken before reaching tout.
At t = 0.00408363, , mxstep steps taken before reaching tout.
At t = 0.00230958, , mxstep steps taken before reaching tout.


print(Sol_ddiff["Electrolyte conductivity [S.m-1]"].entries[:,51] )
print(Sol_ddiff["Electrolyte diffusivity [m2.s-1]"].entries[:,-1])

In [14]:
font = {'family' : 'Times New Roman',
        #'weight' : 'bold',
        'size'   : 18}
mpl.rc('font', **font)

In [40]:
Dcross1 = 5e-14; Dcross2 = 5e-12; Dcross3 = 5e-11; 
label = ["Dcross="+str(Dcross1),"Dcross="+str(Dcross2),"Dcross="+str(Dcross3)] 
output_variables3 = [
    "Terminal voltage [V]",   
    #"Current [A]",
    "EC concentration [mol.m-3]",
    "Electrolyte concentration [mol.m-3]",
    "c(EC) over c(Li+)",
    #"Loss of capacity to SEI [A.h]",
    #"Li+ flux [mol.m-2.s-1]",
    #"EC flux [mol.m-2.s-1]",
    "Electrolyte conductivity [S.m-1]",
    "Electrolyte diffusivity [m2.s-1]",
]
quick_plot = pybamm.QuickPlot([
    Sol_ddiff,Sol_ddiff2,Sol_ddiff3
    ], output_variables3,label,
    variable_limits='fixed',time_unit='hours',n_rows=2,
    figsize = (10,8)) #     spatial_unit='mm',
quick_plot.dynamic_plot();
#quick_plot.create_gif(
#    number_of_images=10, duration=2,output_filename=BasicPath + Target+"fixed_add c_EC as tanh (0-1).gif")

interactive(children=(FloatSlider(value=0.0, description='t', max=4.135324708590893, step=0.041353247085908935…

In [30]:
label = ["Dcross=1.5e-10"] 
output_variables3 = [
    "Terminal voltage [V]",   
    #"Current [A]",
    "EC concentration [mol.m-3]",
    "Electrolyte concentration [mol.m-3]",
    "c(EC) over c(Li+)",
    #"Loss of capacity to SEI [A.h]",
    "Li+ flux [mol.m-2.s-1]",
    "EC flux [mol.m-2.s-1]",
    "Electrolyte conductivity [S.m-1]",
    "Electrolyte diffusivity [m2.s-1]",
]
quick_plot = pybamm.QuickPlot([
    Sol_ddiff6
    ], output_variables3,label,
    variable_limits='fixed',time_unit='hours',n_rows=3,
    figsize = (16,10)) #     spatial_unit='mm',
quick_plot.dynamic_plot();

interactive(children=(FloatSlider(value=0.0, description='t', max=4.160648187256267, step=0.04160648187256267)…